# NextGEOSS
## Invoke the SatCen's Change Detection service using Jupyter

### Inspect the OGC WPS capabilities

In [1]:
from owslib.wps import WebProcessingService
from owslib.wps import monitorExecution
from owslib.wps import printInputOutput

In [12]:
wps_url = "http://change-detection-s1.satcen.nextgeoss.terradue.com/wps/WebProcessingService"

In [5]:
wps = WebProcessingService(wps_url, verbose=False, skip_caps=True)

In [6]:
wps.getcapabilities()

In [7]:
for operation in wps.operations:
    print operation.name

GetCapabilities
DescribeProcess
Execute


In [8]:
for process in wps.processes:
    print process.identifier

com.terradue.wps_oozie.process.OozieAbstractAlgorithm


### Inspect the OGC WPS process information and inputs

In [18]:
process_id = 'com.terradue.wps_oozie.process.OozieAbstractAlgorithm'
process = wps.describeprocess(process_id)

In [19]:
process.title

'Sentinel-1 GRD ACD algorithm'

In [20]:
process.abstract

'Amplitude Change Detection Application on GRD StripMap Sentinel-1 Data'

In [49]:
for input in process.dataInputs:
    printInputOutput(input)
    print '***'

 identifier=source, title=Date of acquisition, abstract=Date of the acquisition that will be taken as master scene in Y-m-d format (e.g. 2018-09-24), data type=string
 Any value allowed
 Default Value: 2018-09-24 
 minOccurs=1, maxOccurs=100
***
 identifier=aoi, title=Area of Interest, abstract=Area of Interest, data type=string
 Any value allowed
 Default Value: 
		POLYGON((8.22762498164556 15.07791004244666,8.763045262446788 12.364797810361907,9.949070337799624 12.826368693010892,10.503161175050785 15.534087438341302,8.22762498164556 15.07791004244666,8.22762498164556 15.07791004244666)))
		 
 minOccurs=1, maxOccurs=1
***
 identifier=polarization, title=Polarization, abstract=Polarization, data type=string
 Any value allowed
 Default Value: VV 
 minOccurs=1, maxOccurs=1
***
 identifier=debug, title=Debug, abstract=Set it to 1 for a more verbose output, data type=string
 Any value allowed
 Default Value: 0 
 minOccurs=1, maxOccurs=1
***


### Prepare the execution request

In [22]:
inputs = []
inputs.append(('source','2018-09-24'))
inputs.append(('aoi','POLYGON((8.22762498164556 15.07791004244666,8.763045262446788 12.364797810361907,9.949070337799624 12.826368693010892,10.503161175050785 15.534087438341302,8.22762498164556 15.07791004244666,8.22762498164556 15.07791004244666)))'))
inputs.append(('polarization','VV'))
inputs.append(('debug','0'))

outputs = [('result_osd',True)]

### Execute and monitor the request

In [23]:
execution = wps.execute(process_id, inputs, outputs)

In [25]:
monitorExecution(execution)

In [26]:
execution.isSucceded()

True

### Access the results' OpenSearch Description (osd)

In [27]:
for output in execution.processOutputs:
    print(output.identifier)

result_osd


In [28]:
results_osd = execution.processOutputs[0].reference

In [29]:
print results_osd

http://change-detection-s1.satcen.nextgeoss.terradue.com/wps/RetrieveResultServlet?id=04104b2a-a605-4122-8b61-33cb61294d97result_osd.493bc4cc-4d54-4dd6-ada7-0d61e9705c59


In [54]:
import requests
r = requests.get(results_osd)
r.text

u'<?xml version=\'1.0\' encoding=\'UTF-8\'?><ns:Reference xmlns:ns="http://www.opengis.net/wps/1.0.0" href="http://change-detection-s1.satcen.nextgeoss.terradue.com/sbws/wps/workflow/0000000-190101000006307-oozie-oozi-W/results/description" method="GET" mimeType="application/opensearchdescription+xml"/>'

In [66]:
import xml.etree.ElementTree as ET
root = ET.fromstring(r.text)
root.attrib['href']

'http://change-detection-s1.satcen.nextgeoss.terradue.com/sbws/wps/workflow/0000000-190101000006307-oozie-oozi-W/results/description'

### Perform an OpenSearch query for the results enclosures

In [30]:
import cioppy

In [31]:
ciop = cioppy.Cioppy()

In [67]:
search = ciop.search(end_point = root.attrib['href'],
                     params = [],
                     output_fields='enclosure')

In [68]:
for result in search:
    print result['enclosure']

http://change-detection-s1.satcen.nextgeoss.terradue.com/data/run/workflow/0000000-190101000006307-oozie-oozi-W/_results/cd_S1B_S3_GRDH_1SDV_20180924T173006_20180924T173034_012860_017BEB_98E0_S1B_S3_GRDH_1SDV_20180912T173005_20180912T173034_012685_017687_5C7A.png
http://change-detection-s1.satcen.nextgeoss.terradue.com/data/run/workflow/0000000-190101000006307-oozie-oozi-W/_results/cd_S1B_S3_GRDH_1SDV_20180924T173006_20180924T173034_012860_017BEB_98E0_S1B_S3_GRDH_1SDV_20180912T173005_20180912T173034_012685_017687_5C7A.pngw
http://change-detection-s1.satcen.nextgeoss.terradue.com/data/run/workflow/0000000-190101000006307-oozie-oozi-W/_results/cd_S1B_S3_GRDH_1SDV_20180924T173006_20180924T173034_012860_017BEB_98E0_S1B_S3_GRDH_1SDV_20180912T173005_20180912T173034_012685_017687_5C7A.tgz
http://change-detection-s1.satcen.nextgeoss.terradue.com/data/run/workflow/0000000-190101000006307-oozie-oozi-W/_results/cd_S1B_S3_GRDH_1SDV_20180924T173034_20180924T173045_012860_017BEB_CA29_S1B_S3_GRDH_1SD

### Download an output file

In [89]:
search[3]['enclosure']

'http://change-detection-s1.satcen.nextgeoss.terradue.com/data/run/workflow/0000000-190101000006307-oozie-oozi-W/_results/cd_S1B_S3_GRDH_1SDV_20180924T173034_20180924T173045_012860_017BEB_CA29_S1B_S3_GRDH_1SDV_20180912T173034_20180912T173045_012685_017687_5033.png'

In [90]:
r = requests.get(search[3]['enclosure'],stream=True)

In [85]:
with open('output.png', 'wb') as fd:
    for chunk in r.iter_content(chunk_size=128):
        fd.write(chunk)